In [7]:
import sys
from ortools.constraint_solver import pywrapcp

def main():
    solver = pywrapcp.Solver('schedule_shifts')
    n_nurses = 4
    n_shifts = 4
    n_days = 7
    
    # creating the shifts decision variable
    shifts = {}
    for nrs in range(n_nurses):
        for day in range(n_days):
            shifts[(nrs, day)] = solver.IntVar(0, n_shifts-1,
                                               "shifts({nrs},{day})".format(nrs=nrs,day=day))
    shifts_flat = [shifts[(nrs, day)] for nrs in range(n_nurses) for day in range(n_days)]
    
    # creating the nurses decision variable
    nurses = {}
    for shft in range(n_shifts):
        for day in range(n_days):
            nurses[(shft, day)] = solver.IntVar(0, n_nurses-1, 
                                               "nurses({shft},{day})".format(shft=shft,day=day))
    
    
    ## Linking shifts and nurses in each day
    for day in range(n_days):
        # go into the nurses variable and lay them out for the day
        nurses_for_day = [nurses[(shft, day)] for shft in range(n_shifts)]
        # go into the shifts variable, and for each shift, link the one \
        # corresponding to the right nurse, to the nurse for the day above
        for nrs in range(n_nurses):
            s = shifts[(nrs, day)]
            solver.Add(s.IndexOf(nurses_for_day)==nrs)
    
    ## CONSTRAINTS ##
    
    # On each day, all nurses are assigned to different shifts
    for day in range(n_days):
        solver.Add(solver.AllDifferent([shifts[(nrs, day)] for nrs in range(n_nurses)]))
        solver.Add(solver.AllDifferent([nurses[(shft, day)] for shft in range(n_shifts)]))

    # Each nurse works five or six days a week.
    for nrs in range(n_nurses):
        solver.Add(solver.Sum([shifts[(nrs, day)]>0 for day in range(n_days)]) >= 5)
        solver.Add(solver.Sum([shifts[(nrs, day)]>0 for day in range(n_days)]) <= 6)

    
    ## No shift is staffed by more than two different nurses in a week.
    works_shift = {}
    for nrs in range(n_nurses):
        for shft in range(n_shifts):
            works_shift[(nrs, shft)] = solver.BoolVar("works_shift({nrs},{shft})".format(nrs=nrs,shft=shft))
    
    for nrs in range(n_nurses):
        for shft in range(n_shifts):
            solver.Add(works_shift[(nrs, shft)] == solver.Max([shifts[(nrs, day)]==shft for day in range(n_days)]))
    
    for shft in range(1, n_shifts):
        solver.Add(solver.Sum([works_shift[(nrs, shft)] for nrs in range(n_nurses)]) <=2)

    # If a nurse works shifts 2 or 3 on a given day, he must also work \
    # the same shift either the previous day or the following day.
    solver.Add(solver.Max(nurses[(2, 0)] == nurses[(2, 1)], nurses[(2, 1)] == nurses[(2, 2)]) == 1)
    solver.Add(solver.Max(nurses[(2, 1)] == nurses[(2, 2)], nurses[(2, 2)] == nurses[(2, 3)]) == 1)
    solver.Add(solver.Max(nurses[(2, 2)] == nurses[(2, 3)], nurses[(2, 3)] == nurses[(2, 4)]) == 1)
    solver.Add(solver.Max(nurses[(2, 3)] == nurses[(2, 4)], nurses[(2, 4)] == nurses[(2, 5)]) == 1)
    solver.Add(solver.Max(nurses[(2, 4)] == nurses[(2, 5)], nurses[(2, 5)] == nurses[(2, 6)]) == 1)
    solver.Add(solver.Max(nurses[(2, 5)] == nurses[(2, 6)], nurses[(2, 6)] == nurses[(2, 0)]) == 1)
    solver.Add(solver.Max(nurses[(2, 6)] == nurses[(2, 0)], nurses[(2, 0)] == nurses[(2, 1)]) == 1)
    solver.Add(solver.Max(nurses[(3, 0)] == nurses[(3, 1)], nurses[(3, 1)] == nurses[(3, 2)]) == 1)
    solver.Add(solver.Max(nurses[(3, 1)] == nurses[(3, 2)], nurses[(3, 2)] == nurses[(3, 3)]) == 1)
    solver.Add(solver.Max(nurses[(3, 2)] == nurses[(3, 3)], nurses[(3, 3)] == nurses[(3, 4)]) == 1)
    solver.Add(solver.Max(nurses[(3, 3)] == nurses[(3, 4)], nurses[(3, 4)] == nurses[(3, 5)]) == 1)
    solver.Add(solver.Max(nurses[(3, 4)] == nurses[(3, 5)], nurses[(3, 5)] == nurses[(3, 6)]) == 1)
    solver.Add(solver.Max(nurses[(3, 5)] == nurses[(3, 6)], nurses[(3, 6)] == nurses[(3, 0)]) == 1)
    solver.Add(solver.Max(nurses[(3, 6)] == nurses[(3, 0)], nurses[(3, 0)] == nurses[(3, 1)]) == 1)
    
    
    #### TIM TO SOLVE ###
    db = solver.Phase(shifts_flat,
                     solver.CHOOSE_FIRST_UNBOUND,
                     solver.ASSIGN_MIN_VALUE)
    
    solution = solver.Assignment()
    solution.Add(shifts_flat)
    collector = solver.AllSolutionCollector(solution)
    
    solver.Solve(db, [collector])
    
    n_solutions = collector.SolutionCount()
    print("Time:", solver.WallTime(), "ms")
    print("Solutions found:", n_solutions)
    
    if n_solutions > 0:   
        #Display a few solutions picked at random.
        a_few_solutions = [6, 66, 666]

        for sol in a_few_solutions:
            print("Solution number" , sol, '\n')

            for i in range(n_days):
                print("Day", i)
                for j in range(n_nurses):
                    print("Nurse", j, "assigned to task",
                          collector.Value(sol, shifts[(j, i)]))
    else:
        print('oops')

if __name__ == "__main__":
    main()

Time: 707 ms
Solutions found: 18144
Solution number 6 

Day 0
Nurse 0 assigned to task 0
Nurse 1 assigned to task 1
Nurse 2 assigned to task 3
Nurse 3 assigned to task 2
Day 1
Nurse 0 assigned to task 0
Nurse 1 assigned to task 1
Nurse 2 assigned to task 3
Nurse 3 assigned to task 2
Day 2
Nurse 0 assigned to task 1
Nurse 1 assigned to task 0
Nurse 2 assigned to task 3
Nurse 3 assigned to task 2
Day 3
Nurse 0 assigned to task 1
Nurse 1 assigned to task 0
Nurse 2 assigned to task 3
Nurse 3 assigned to task 2
Day 4
Nurse 0 assigned to task 1
Nurse 1 assigned to task 3
Nurse 2 assigned to task 0
Nurse 3 assigned to task 2
Day 5
Nurse 0 assigned to task 1
Nurse 1 assigned to task 3
Nurse 2 assigned to task 2
Nurse 3 assigned to task 0
Day 6
Nurse 0 assigned to task 1
Nurse 1 assigned to task 3
Nurse 2 assigned to task 2
Nurse 3 assigned to task 0
Solution number 66 

Day 0
Nurse 0 assigned to task 0
Nurse 1 assigned to task 2
Nurse 2 assigned to task 3
Nurse 3 assigned to task 1
Day 1
Nurse

## Job shop

In [20]:
from ortools.constraint_solver import pywrapcp

def main():
    n_machines = 3
    
   
    job0 = [(0, 3), (1, 2), (2, 2)]
    job1 = [(0, 2), (2, 1), (1, 4)]
    job2 = [(1, 4), (2, 3)]
    all_tasks_list = [job0, job1, job2]
    n_jobs = len(all_tasks_list)
    
    machines = [[all_tasks_list[job][tsk][0] for tsk in range(len(all_tasks_list[job]))] for job in range(n_jobs)]    
    processing_times = [[all_tasks_list[job][tsk][1] for tsk in range(len(all_tasks_list[job]))] for job in range(n_jobs)]    
    
    solver = pywrapcp.Solver('jobshop')
    
    # computes horizon
    horizon = 0
    for job in range(n_jobs):
        horizon += sum(processing_times[job])
        
    #### CREATE THE tasks IN THE SOLVER ###
    # (has no knowledge of machines)
    all_tasks = {}
    for job in range(n_jobs):
        for tsk in range(len(all_tasks_list[job])):
            all_tasks[(job, tsk)] = solver.FixedDurationIntervalVar(0,
                                                                    horizon,
                                                                    processing_times[job][tsk],
                                                                    False,
                                                                   'some_name')
    
    # creat the disjunctive constraints \
    # (no machine work on one job until the previous is done)
    # the fixed Duration Interval Vars should be disjunctive
    all_sequences = []
    all_machines_jobs = []
    for mch in range(n_machines):
        
        machine_jobs = []
        for job in range(n_jobs):
            for tsk in range(len(all_tasks_list[job])):
                if machine[job][tsk] == mch:
                    machines_jobs.append(all_tasks[(job,tsk)])
        
        # the time intervals vas can't overlap for that machine's jobs
        disj = solver.DisjunctiveConstraint(machine_jobs)
        solver.Add(disj)
        all_sequences.append(disj.SequenceVar())
        
    ## Add the conjuctive constraints (certain jobs must follow each other)
    for job in range(n_jobs):
        for tsk in range(len(all_tasks_list[job])-1):
            solver.Add(all_tasks[(job, tsk+1)].SartsAfterEnd(all_tasks[(job, tsk)]))
    
    
    # objective Function 
    
    objective_value = solver.Max([all_tasks[(job, len(all_tasks_list[job])-1)].EndExpr()
                                  for job in range(n_jobs)])
    
    objective = solver.Minimize(objective_value,1)
    
    sequence_phase = solver.Phase([all_sequences[mch] for mch in range(n_machines)],
                                 solver.SEQUENCE_DEFAULT)
    
    vars_phase = solver.Phase([obj_var],
                              solver.CHOOSE_FIRST_UNBOUND,
                              solver.ASSIGN_MIN_VALUE)
    
    full_phase = solver.Compose([sequence_phase, vars_phase])
    
    collector = solver.LastSolutionCollector()
    
    collector.Add(all_sequences)
    collector.AddObjective(obj_var)
    
    for mch in range(n_machines):
        sequence = all_sequences[mch]
        sequence_count = sequence.Size()
        for j in range(sequence_count):
            t = sequence.Interval(j)
            collector.Add(t.StartExpr().Var())
            collector.Add(t.EndExpr().Var())
    
    disp_col_width = 10
    if solver.Solve(main_phase, [objective_monitor, collector]):
        print("\nOptimal Schedule Length:", collector.ObjectiveValue(0), "\n")
        sol_line = ""
        sol_line_tasks = ""
        print("Optimal Schedule", "\n")
        for mch in range(n_machines):
            seq = all_sequences[i]
            sol_line += "Machine " + str(mch) + ": "
            sol_line_tasks += "Machine " + str(mch) + ": "
            sequence = collector.ForwardSequence(0, seq)
        seq_size = len(sequence)
        
        for j in range(0, seq_size):
            t = seq.Interval(sequence[j]);
            # Add spaces to output to align columns.
            sol_line_tasks +=  t.Name() + " " * (disp_col_width - len(t.Name()))

        for j in range(0, seq_size):
            t = seq.Interval(sequence[j]);
            sol_tmp = "[" + str(collector.Value(0, t.StartExpr().Var())) + ","
            sol_tmp += str(collector.Value(0, t.EndExpr().Var())) + "] "
            # Add spaces to output to align columns.
            sol_line += sol_tmp + " " * (disp_col_width - len(sol_tmp))

        sol_line += "\n"
        sol_line_tasks += "\n"

    print(sol_line_tasks)
    print("Time Intervals for Tasks\n")
    print(sol_line)

if __name__ == '__main__':
    main()

NameError: name 'machine' is not defined

In [9]:

    
  # Solve the problem.
  disp_col_width = 10
  if solver.Solve(main_phase, [objective_monitor, collector]):
    print("\nOptimal Schedule Length:", collector.ObjectiveValue(0), "\n")
    sol_line = ""
    sol_line_tasks = ""
    print("Optimal Schedule", "\n")

    for i in all_machines:
      seq = all_sequences[i]
      sol_line += "Machine " + str(i) + ": "
      sol_line_tasks += "Machine " + str(i) + ": "
      sequence = collector.ForwardSequence(0, seq)
      seq_size = len(sequence)

      for j in range(0, seq_size):
        t = seq.Interval(sequence[j]);
         # Add spaces to output to align columns.
        sol_line_tasks +=  t.Name() + " " * (disp_col_width - len(t.Name()))

      for j in range(0, seq_size):
        t = seq.Interval(sequence[j]);
        sol_tmp = "[" + str(collector.Value(0, t.StartExpr().Var())) + ","
        sol_tmp += str(collector.Value(0, t.EndExpr().Var())) + "] "
        # Add spaces to output to align columns.
        sol_line += sol_tmp + " " * (disp_col_width - len(sol_tmp))

      sol_line += "\n"
      sol_line_tasks += "\n"

    print(sol_line_tasks)
    print("Time Intervals for Tasks\n")
    print(sol_line)

if __name__ == '__main__':
  main()


Optimal Schedule Length: 11 

Optimal Schedule 

Machine 0: Job_0_0   Job_1_0   
Machine 1: Job_2_0   Job_0_1   Job_1_1   
Machine 2: Job_2_1   Job_0_2   Job_1_2   

Time Intervals for Tasks

Machine 0: [0,3]     [3,5]     
Machine 1: [0,4]     [4,6]     [6,10]    
Machine 2: [4,7]     [7,9]     [10,11]   

